In [ ]:
import orbslam2
import time

vocab_path = "Vocabulary/ORBvoc.txt" 
#settings_path = "Calibration/TUM3.yaml"
#dataset_path = "rgbd_dataset_freiburg3_long_office_household_validation/rgb/"
settings_path = "CameraParams/JetBot960.yaml"
dataset_path = "Datasets/jetbot_dataset_960/rgb/"

In [ ]:
slam = orbslam2.System(vocab_path, settings_path, orbslam2.Sensor.MONOCULAR)
slam.set_use_viewer(False)
slam.initialize()
print("Successfully initialize ORB-SLAM.")

In [ ]:
def update(img):
    global slam, starttime, map_points 
    timestemp = time.time() - starttime
    slam.process_image_mono(img, float(timestemp))
    
    # Get Tracking State
    track_state = slam.get_tracking_state()
    print('\rTimestemp:', str(timestemp)[0:5], "| State:", track_state, end='')
    
    # Get Pose and Map Information
    trajectory = slam.get_trajectory_points()
    if len(trajectory) == 0:
        loc = None
        tracked_points = {}
    else:
        tracked_points = slam.get_tracked_mappoints()
        map_points.update(tracked_points)
        trans = slam.get_trajectory_points()[-1]
        loc = (round(trans[4], 4), round(trans[8], 4), round(trans[12], 4))
        
    print(" | Location:", loc, "| Tracked Map Size:", len(tracked_points), "| Map Size:", len(map_points), end='')

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import os
import cv2

# Activate jetbot camera and link it to the display
image = widgets.Image(format='jpeg', width=320, height=240) 
display(image)

map_points = {}
starttime = time.time()
slam.reset()

images = sorted(os.listdir(dataset_path))
for i, fname in enumerate(images):
    if fname.split(".")[-1] != "jpg" and fname.split(".")[-1] != "png":
        continue
    img_file = open(dataset_path + fname, "rb")
    image.value = img_file.read()
    img = cv2.imread(dataset_path+fname)
    update(img)
    time.sleep(0.01)
    if i > 500:
        break

In [ ]:
import numpy as np
import json

# Save Trajectory
traj_pts = slam.get_trajectory_points()
timestemps = np.array([t[0] for t in traj_pts])
transforms = np.array([t[1:13] for t in traj_pts]).reshape(-1,3,4)
np.savez("dataset_poses.npz", timestemps=timestemps, transforms=transforms)

# Save Map Points
file = open("dataset_map_points.json", "w")
json.dump(map_points, file)
file.close()

In [ ]:
import numpy as np
import json

# Load Poses
rec = np.load('dataset_poses.npz')
timestemps, transforms = rec['timestemps'], rec['transforms']

# Load Map Points
file = open("dataset_map_points.json", "r")
map_points = json.load(file)
file.close()

In [ ]:
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

print("Plot Map Points ...")
x_mpts, y_mpts, z_mpts = [], [], []
for mid in map_points:
    x_mpts.append(map_points[mid][0])
    y_mpts.append(map_points[mid][1])
    z_mpts.append(map_points[mid][2])

mpts = go.Scatter3d(
    x=x_mpts, y=y_mpts, z=z_mpts,  
    mode='markers', #'markers','lines','lines+markers'
    marker={'size': 2, 'opacity': 0.8, 'color':'rgb(100,100,100)'},
)

print("Plot Location ...")
x_traj, y_traj, z_traj = [], [], []
for i in range(transforms.shape[0]):
    x_traj.append(transforms[i,0,3])
    y_traj.append(transforms[i,1,3])
    z_traj.append(transforms[i,2,3])
    
loc = go.Scatter3d(
    x=x_traj, y=y_traj, z=z_traj,  
    mode='markers', #'markers','lines','lines+markers'
    marker={'size': 3, 'opacity': 0.8},
)

print("Plot Trajectory ...")
traj = go.Scatter3d(
    x=x_traj, y=y_traj, z=z_traj,  
    mode='lines', #'markers','lines','lines+markers'
    line=dict(color='rgb(255, 100, 255)',width=3),
)
data = [loc, traj, mpts]

# Frame Axis
print("Plot Axis ...")
scale = 0.02
for i in range(0,transforms.shape[0],1):
    print("\rPlot axis of frame", i, end="")
    trans = transforms[i]
    x, y, z = trans[0,3], trans[1,3], trans[2,3]
    axis_x = go.Scatter3d(x=[x,x+scale*trans[0,0]],y=[y,y+scale*trans[1,0]],z=[z,z+scale*trans[2,0]],mode='lines',line=dict(color='#ff0000',width=4))
    axis_y = go.Scatter3d(x=[x,x+scale*trans[0,1]],y=[y,y+scale*trans[1,1]],z=[z,z+scale*trans[2,1]],mode='lines',line=dict(color='#00ff00',width=4))
    axis_z = go.Scatter3d(x=[x,x+scale*trans[0,2]],y=[y,y+scale*trans[1,2]],z=[z,z+scale*trans[2,2]],mode='lines',line=dict(color='#0000ff',width=4))
    data += [axis_x, axis_y, axis_z]

# Configure the layout.
print("\nBuild Figure ...")
layout = go.Layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 0}, scene=dict(aspectmode="data"))
plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
print("Rendering ...")
plotly.offline.iplot(plot_figure)